1. Plan travel using resrobot route planner
  - a) Find the stops names between Göteborg and Lund for a train that is departuring from Göteborg.

  - b) Find stops names, and times between two different cities of your choice in Sweden.

  - c) See if you can plot in a map points of a train on each stops between two locations of your choice.

2. Make digital display boards for trains/trams
  - a) Make a departure board for trams and buses for a station of your choice, e.g. Göteborg korsvägen. It displays how many minutes are left to wait. An example could be to print out

spårvagn 8 mot angered 2 minuter
spårvagn 2 mot högsbohöjd 5 minuter
...
  - b) Make a departure board for trains. Here list the trains, their destinations and the time it departures.

  - c) Make an arrival board for trains. Here list the trains, their destinations and the time it arrives.

In [ ]:
from dotenv import load_dotenv
import os 
import requests


load_dotenv() # läser in miljövariablen från .env filen 

API_KEY = os.getenv("API_KEY") # hämtar värdet från nyckeln 



url = f"https://api.resrobot.se/v2.1/departureBoard?id=740000002&format=json&accessId={API_KEY}" # id = göteborgs centralstation, formatet är json och min api nyckel

response = requests.get(url)

# print(response) -- Detta gav svar <Response [200]> vilket menas att api anropet lyckades.

data = response.json() # detta konventerar datan från json till python objektformat i detta fall en dict.

# Extrahera alla avgångar
departures = data.get("Departure", [])[:5] # Hämtar listan med avgångar (5 första!)

# Iterera genom avgångarna och skriv ut relevanta detaljer
for departure in departures:
    line_name = departure.get("ProductAtStop", {}).get("name")  # Namn på linjen
    operator = departure.get("ProductAtStop", {}).get("operator")  # Operatör
    time = departure.get("time")  # Avgångstid
    print(f"Linje: {line_name}, Operatör: {operator}, Avgångstid: {time}")




Linje: Länstrafik - Buss 21, Operatör: Västtrafik, Avgångstid: 12:12:00
Linje: Länstrafik - Buss 21, Operatör: Västtrafik, Avgångstid: 12:13:00
Linje: Länstrafik - Buss X4, Operatör: Västtrafik, Avgångstid: 12:14:00
Linje: Länstrafik - Buss X1, Operatör: Västtrafik, Avgångstid: 12:14:00
Linje: Länstrafik - Buss 513, Operatör: Västtrafik, Avgångstid: 12:14:00


In [4]:
#  a) Find the stops names between Göteborg and Lund for a train that is departuring from Göteborg.
location = "stockholm"
#stop lookup api anrop
url = f"https://api.resrobot.se/v2.1/location.name?input={location}&format=json&accessId={API_KEY}" # hämtar här Stop eller extID för stationerna. just nu inställd på göteborg vilket ger extId': '740098001' Lund gav 740098038

response = requests.get(url)

loc = response.json()

loc

{'stopLocationOrCoordLocation': [{'StopLocation': {'productAtStop': [{'icon': {'res': 'prod_gen'},
      'cls': '2'},
     {'icon': {'res': 'prod_gen'}, 'cls': '4'},
     {'icon': {'res': 'prod_gen'}, 'cls': '8'},
     {'icon': {'res': 'prod_gen'}, 'cls': '16'},
     {'icon': {'res': 'prod_gen'}, 'cls': '32'},
     {'icon': {'res': 'prod_gen'}, 'cls': '64'},
     {'icon': {'res': 'prod_gen'}, 'cls': '128'}],
    'timezoneOffset': 60,
    'id': 'A=1@O=STOCKHOLM@X=18058151@Y=59330136@U=1@L=740098000@B=1@p=1736935053@',
    'extId': '740098000',
    'name': 'STOCKHOLM',
    'lon': 18.058151,
    'lat': 59.330136,
    'weight': 32767,
    'products': 254,
    'meta': True,
    'minimumChangeDuration': 'PT26M'}},
  {'StopLocation': {'productAtStop': [{'icon': {'res': 'prod_gen'},
      'cls': '2'},
     {'icon': {'res': 'prod_gen'}, 'cls': '4'},
     {'icon': {'res': 'prod_gen'}, 'cls': '8'},
     {'icon': {'res': 'prod_gen'}, 'cls': '16'},
     {'icon': {'res': 'prod_gen'}, 'cls': '32'},
 

In [5]:
#  a) Find the stops names between Göteborg and Lund for a train that is departuring from Göteborg.

# Routeplanner API-anrop
url = f"https://api.resrobot.se/v2.1/trip?format=json&originId=740000002&destId=740098038&passlist=1&showPassingPoints=true&accessId={API_KEY}"  # passlist = 1 inkluderar en lista med alla passerade stopp.

response = requests.get(url)  # Gör API-anropet.

stops = response.json()  # Konverterar svaret från JSON till en Python-dictionary.

trip = stops.get("Trip", [])[0]  # Hämtar den första resan i resultatet från anropet.

pass_list = trip.get("LegList", {}).get("Leg", [])  # Hämtar alla delar (legs) av resan.

for leg in pass_list:
    stoppen = leg.get("Stops", {}).get("Stop", [])  # Hämtar alla stopp för detta ben av resan.
    for stop in stoppen:
        stop_name = stop.get("name")  # Hämtar namnet på stoppet.
        times = stop.get("arrTime")
        print(f"Stop: {stop_name} Time: {times}")  


Stop: Göteborg Centralstation Time: None
Stop: Mölndal station Time: 12:48:00
Stop: Kungsbacka station Time: 12:59:00
Stop: Varberg station Time: 13:25:00
Stop: Falkenberg station Time: 13:45:00
Stop: Halmstad Centralstation Time: 14:03:00
Stop: Laholm station Time: 14:25:00
Stop: Båstad station Time: 14:32:00
Stop: Ängelholm station Time: 14:43:00
Stop: Helsingborg Centralstation Time: 15:04:00
Stop: Landskrona station Time: 15:19:00
Stop: Lund Centralstation Time: 15:36:00


In [6]:
#  b) Find stops names, and times between two different cities of your choice in Sweden.

extid_1 = 740000001 # sthlm
extid_2 = 740000003 # malmö

def route_planner (dep_id, arr_id):
    url = f"https://api.resrobot.se/v2.1/trip?format=json&originId={dep_id}&destId={arr_id}&passlist=1&showPassingPoints=true&accessId={API_KEY}"

    response = requests.get(url)  # Gör API-anropet.

    stops = response.json()  # Konverterar svaret från JSON till en Python-dictionary.

    trip = stops.get("Trip", [])[0]  # Hämtar den första resan i resultatet från anropet.

    pass_list = trip.get("LegList", {}).get("Leg", [])  # Hämtar alla delar (legs) av resan.

    stops_and_times = [] # Lista för att lagra alla stopp och tider

    for leg in pass_list:
        stoppen = leg.get("Stops", {}).get("Stop", [])  # Hämtar alla stopp för detta ben av resan.
        for stop in stoppen:
            stop_name = stop.get("name")  # Hämtar namnet på stoppet.
            times = stop.get("arrTime")
            stops_and_times.append((stop_name, times))  # Lägg till tuple i listan
    
    
    return stops_and_times

test = route_planner(extid_1, extid_2)


for stop_name, times in test:  # Iterera genom varje tuple i listan
    print(f"Stopp: {stop_name}, Tid: {times}")




Stopp: Stockholm Centralstation, Tid: None
Stopp: Sundbyberg station, Tid: None
Stopp: Bålsta station (Håbo kn), Tid: 12:41:00
Stopp: Enköping station, Tid: 12:54:00
Stopp: Västerås Centralstation, Tid: 13:09:00
Stopp: Köping station, Tid: 13:28:00
Stopp: Arboga station, Tid: 13:39:00
Stopp: Örebro Centralstation, Tid: 14:01:00
Stopp: Örebro Södra station, Tid: 14:17:00
Stopp: Kumla station, Tid: 14:27:00
Stopp: Hallsberg station, Tid: 14:35:00
Stopp: Laxå station, Tid: 14:57:00
Stopp: Töreboda station, Tid: 15:20:00
Stopp: Skövde Centralstation, Tid: 15:35:00
Stopp: Falköping Centralstation, Tid: 15:51:00
Stopp: Herrljunga station, Tid: 16:06:00
Stopp: Vårgårda station, Tid: 16:14:00
Stopp: Alingsås station, Tid: 16:24:00
Stopp: Göteborg Centralstation, Tid: 17:00:00
Stopp: Göteborg Centralstation, Tid: None
Stopp: Mölndal station, Tid: 17:48:00
Stopp: Kungsbacka station, Tid: 17:59:00
Stopp: Varberg station, Tid: 18:25:00
Stopp: Falkenberg station, Tid: 18:45:00
Stopp: Halmstad Centr

In [7]:
#  a) Find the stops names between Göteborg and Lund for a train that is departuring from Göteborg.

start_id = 740000001
end_id = 740000003

# Routeplanner API-anrop
url = f"https://api.resrobot.se/v2.1/trip?format=json&originId={start_id}&destId={end_id}&passlist=1&showPassingPoints=true&accessId={API_KEY}"  # passlist = 1 inkluderar en lista med alla passerade stopp.

response = requests.get(url)  # Gör API-anropet.

stops = response.json()  # Konverterar svaret från JSON till en Python-dictionary.

trip = stops.get("Trip", [])[0]  # Hämtar den första resan i resultatet från anropet.

pass_list = trip.get("LegList", {}).get("Leg", [])  # Hämtar alla delar (legs) av resan.

print(f"{'Stop Name':<30} {'Time':<10} {'Lat':<15} {'Lon':<15}")
for leg in pass_list:
    stoppen = leg.get("Stops", {}).get("Stop", [])  # Hämtar alla stopp för detta ben av resan.
    for stop in stoppen:
        stop_name = stop.get("name")  # Hämtar namnet på stoppet.
        times = stop.get("arrTime")
        lat = stop.get("lat")
        lon = stop.get("lon")
        times_str = times if times is not None else "N/A"
        print(f"{stop_name:<30} {times_str:<10} {lat:<15} {lon:<15}")

Stop Name                      Time       Lat             Lon            
Stockholm Centralstation       N/A        59.330136       18.058151      
Sundbyberg station             N/A        59.361032       17.970938      
Bålsta station (Håbo kn)       12:41:00   59.568754       17.532813      
Enköping station               12:54:00   59.644327       17.086741      
Västerås Centralstation        13:09:00   59.607705       16.551712      
Köping station                 13:28:00   59.506675       16.004215      
Arboga station                 13:39:00   59.397105       15.841061      
Örebro Centralstation          14:01:00   59.278942       15.211348      
Örebro Södra station           14:17:00   59.269989       15.202332      
Kumla station                  14:27:00   59.126377       15.14028       
Hallsberg station              14:35:00   59.066698       15.110391      
Laxå station                   14:57:00   58.989391       14.616658      
Töreboda station               15:20:0

In [92]:
def access_id_from_location(location):
    url = f"https://api.resrobot.se/v2.1/location.name?input={location}&format=json&accessId={API_KEY}"

    try:
        response = requests.get(url)
        result = response.json()

        print(f"{'Name':<50} extId")

        for stop in result.get("stopLocationOrCoordLocation"):
            stop_data = next(iter(stop.values()))

            # returns None if extId doesn't exist
            if stop_data.get("extId"):
                print(f"{stop_data['name']:<50} {stop_data['extId']}")

    except requests.exceptions.RequestException as err:
        print(f"Network or HTTP error: {err}")

access_id_from_location("göteborg")

Name                                               extId
GÖTEBORG                                           740098001
Göteborg Centralstation                            740000002
Göteborg Sävenäs lokstation                        740016365
GÖTEBORG GAMLESTADEN                               740098526
Göteborg Korsvägen                                 740015578
Göteborg Kungsportsplatsen                         740016358
Göteborg Stenpiren                                 740072430
Göteborg Eketrägatan                               740025624
Göteborg Vårväderstorget                           740025707
Göteborg Axel Dahlströms torg                      740025608


In [ ]:
def routplan_for_map(start_id, end_id):
    url = f"https://api.resrobot.se/v2.1/trip?format=json&originId={start_id}&destId={end_id}&passlist=1&showPassingPoints=true&accessId={API_KEY}"  # passlist = 1 inkluderar en lista med alla passerade stopp.

    response = requests.get(url)  # Gör API-anropet.

    stops = response.json()  # Konverterar svaret från JSON till en Python-dictionary.

    trip = stops.get("Trip", [])[0]  # Hämtar den första resan i resultatet från anropet.

    pass_list = trip.get("LegList", {}).get("Leg", [])  # Hämtar alla delar (legs) av resan.

    data = []

    for leg in pass_list:
        stoppen = leg.get("Stops", {}).get("Stop", [])  # Hämtar alla stopp för detta ben av resan.
        for stop in stoppen:
            stop_name = stop.get("name")  # Hämtar namnet på stoppet.
            times = stop.get("arrTime")
            lat = stop.get("lat")
            lon = stop.get("lon")
            times_str = times if times is not None else "N/A"
            lat_str = lat if lat is not None else "N/A"
            lon_str = lon if lon is not None else "N/A"
            data.append((stop_name, times_str, lat_str, lon_str))
    return data

trying = routplan_for_map(start_id =740000001, end_id =740000003 )
print(f"{'Stop Name':<30} {'Time':<10} {'Lat':<15} {'Lon':<15}")
for t in trying:
    print(f"{t[0]:<30} {t[1]:<10} {t[2]:<15} {t[3]:<15}")


In [12]:
# c) See if you can plot in a map points of a train on each stops between two locations of your choice.

import folium 

# Skapa en karta centrerad på Stockholm Centralstation (första stoppet)
mapzon = folium.Map(location=(59.330136, 18.058151), zoom_start=7)

# Lägg till en markör för startpunkten (Stockholm Centralstation)
# folium.Marker(location=(59.330136, 18.058151), popup="Stockholm Centralstation", tooltip="Start av resan").add_to(mapzon)

# Iterera genom stoppen och lägg till markörer för varje stopp
for stop_name, times, lat, lon in trying:
    folium.Marker(
        location=(lat, lon),  # Koordinater för stoppet
        popup=f"{stop_name}<br>Tid: {times}",  # Visa namn och tid i popup
        tooltip=stop_name,  # Tooltip med stoppets namn
    ).add_to(mapzon)

# Spara kartan som en HTML-fil
mapzon.save("index.html")




In [ ]:
import datetime
# a) Make a departure board for trams and buses for a station of your choice, e.g. Göteborg korsvägen. It displays how many minutes are left to wait. An example could be to print out

ullevi_norra_extid = 740025695

url = f"https://api.resrobot.se/v2.1/departureBoard?id={ullevi_norra_extid}&format=json&accessId={API_KEY}"

response = requests.get(url)  # Gör API-anropet.

dep = response.json()  # Konverterar svaret från JSON till en Python-dictionary.

avgång = dep.get("Departure", [])[:5]

# Hämta nuvarande tid
now = datetime.datetime.now()

print(f"{'Linje':<30}{'Mot':30}{'Avgång om':<30}")
print("-" * 80)

for avgångar in avgång:
    avgång_ = avgångar.get("ProductAtStop", {}).get("name", "")  # Namn på linjen
    avgång_ = avgång_.replace("Länstrafik - ", "")  # Ta bort "Länstrafik - "
    
    time_ = avgångar.get("time")  # Avgångstid som HH:MM:SS
    direction_ = avgångar.get("direction")  # Ändhållplats

    # Konvertera avgångstiden till ett datetime-objekt
    dagens_datum = now.date()
    try:
        avgång_tid = datetime.datetime.combine(dagens_datum, datetime.datetime.strptime(time_, "%H:%M:%S").time())  # Kombinerar dagens datum (dagens_datum) med avgångstiden (time_)
        # - datetime.datetime.strptime(time_, "%H:%M:%S").time(): Konverterar avgångstiden från en sträng i formatet "HH:MM:SS" till ett time-objekt.
        # - datetime.datetime.combine(): Lägger till dagens datum till detta time-objekt och skapar ett komplett datetime-objekt (datum + tid).
        # Resultatet är ett datetime-objekt som representerar det exakta datumet och tiden för avgången.
    except ValueError:
        continue  # Om tiden inte är korrekt formaterad, hoppa över avgången

    # Beräkna skillnaden i minuter
    time_diff = avgång_tid - now
    minutes_left = int(time_diff.total_seconds() // 60)

    # Visa bara avgångar som är i framtiden
    if minutes_left >= 0:
        print(f"{avgång_:<20}   {direction_:<30}        {minutes_left:<5}min")

Linje                         Mot                           Avgång om                     
--------------------------------------------------------------------------------
Spårväg 1              Göteborg Östra sjukhuset              1    min
Spårväg 6              Göteborg Varmfrontsgatan              2    min
Spårväg 8              Angered centrum (Göteborg kn)         3    min
Spårväg 3              Virginsgatan (Göteborg kn)            4    min
Spårväg 1              Tynnered Opaltorget (Göteborg kn)        4    min


In [137]:
# b) Make a departure board for trains. Here list the trains, their destinations and the time it departures.

def train_departure_board(station_id, api_key):
    """
    Hämtar och skriver ut en avgångstavla för tåg från en vald station.
    """
    url = f"https://api.resrobot.se/v2.1/departureBoard?id={station_id}&format=json&accessId={api_key}"
    response = requests.get(url)

    if response.status_code != 200:
        print(f"API-anrop misslyckades med statuskod {response.status_code}")
        print(response.text)
        return

    data = response.json()
    departures = data.get("Departure", [])

    print(f"{'Train':<27}{'Destination':<30}{'Departure Time':<20}")
    print("-" * 70)

    for departure in departures:
        cat_out_l = departure.get("ProductAtStop", {}).get("catOutL", "")
        
        
        # Filtrera för tåg baserat på catOutL 
        if cat_out_l not in ["Länstrafik - Tåg", "Regional Tåg", "Snabbtåg"]:
            continue

        product_name = departure.get("ProductAtStop", {}).get("name", "N/A")
        direction = departure.get("direction", "N/A")
        departure_time = departure.get("time", "N/A")

        print(f"{product_name:<25} {direction:<35} {departure_time:<20}")


# Använd funktionen

GOTEBORG_CENTRAL_ID = 740000002  # Göteborg Centralstation
train_departure_board(GOTEBORG_CENTRAL_ID, API_KEY)




Train                      Destination                   Departure Time      
----------------------------------------------------------------------
Länstrafik - Tåg 3258     Öxnered station (Vänersborg kn)     14:30:00            
Länstrafik - Tåg 3109     Kungsbacka station                  14:30:00            
Länstrafik - Tåg 3758     Strömstad station                   14:40:00            
Regional Tåg 1111         Köpenhamn Österport (Danmark)       14:40:00            
Länstrafik - Tåg 3059     Kungsbacka station                  14:45:00            
Länstrafik - Tåg 3658     Älvängen station (Ale kn)           14:50:00            
Länstrafik - Tåg 3260     Öxnered station (Vänersborg kn)     15:00:00            
Länstrafik - Tåg 3111     Kungsbacka station                  15:00:00            
Regional Tåg 182          Stockholm Centralstation            15:02:00            
Länstrafik - Tåg 13608    Älvängen station (Ale kn)           15:05:00            
Länstrafik - Tåg 3560

In [ ]:
def train_arrival_board(station_id, api_key):
    """
    Skapar en ankomsttavla för tåg för en specifik station.
    """
    url = f"https://api.resrobot.se/v2.1/arrivalBoard?id={station_id}&format=json&accessId={api_key}"
    response = requests.get(url)

    if response.status_code != 200:
        print(f"API-anrop misslyckades med statuskod {response.status_code}")
        print(response.text)
        return

    data = response.json()
    arrivals = data.get("Arrival", [])  # Hämta ankomster från svaret

    print(f"{'Train':<27}{'Arriving from':<30}{'Arrival Time':<20}")
    print("-" * 70)

    for arrival in arrivals:
        cat_out_l = arrival.get("ProductAtStop", {}).get("catOutL", "")

        # Filtrera för tåg baserat på catOutL 
        if cat_out_l not in ["Länstrafik - Tåg", "Regional Tåg", "Snabbtåg"]:
            continue

        product_name = arrival.get("ProductAtStop", {}).get("name", "N/A")
        origin = arrival.get("origin", "N/A")  
        arrival_time = arrival.get("time", "N/A")  # Ankomsttid finns som 'time'

        print(f"{product_name:<25} {origin:<35} {arrival_time:<20}")

# Använd funktionen
GOTEBORG_CENTRAL_ID = 740000002  # Göteborg Centralstation
train_arrival_board(GOTEBORG_CENTRAL_ID, API_KEY)




Train                      Arriving from                 Arrival Time        
----------------------------------------------------------------------
Regional Tåg 363          Karlstad Centralstation             14:40:00            
Snabbtåg 431              Stockholm Centralstation            14:41:00            
Länstrafik - Tåg 3056     Kungsbacka station                  14:45:00            
Länstrafik - Tåg 3553     Alingsås station                    14:54:00            
Regional Tåg 20162        Varberg station                     14:55:00            
Länstrafik - Tåg 3655     Älvängen station (Ale kn)           14:55:00            
Regional Tåg 169          Stockholm Centralstation            14:57:00            
Länstrafik - Tåg 3253     Öxnered station (Vänersborg kn)     15:00:00            
Länstrafik - Tåg 3108     Kungsbacka station                  15:00:00            
Länstrafik - Tåg 3058     Kungsbacka station                  15:15:00            
Länstrafik - Tåg 3755